In [99]:
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, DateType
from pyspark import SparkContext, SparkConf

import os

In [100]:
spark_session = SparkSession.builder.appName("Lab2").getOrCreate()
sc = spark_session._sc
spark_session

In [101]:
!head -n 4 /mnt/data/programming-languages.csv

name,wikipedia_url
A# .NET,https://en.wikipedia.org/wiki/A_Sharp_(.NET)
A# (Axiom),https://en.wikipedia.org/wiki/A_Sharp_(Axiom)
A-0 System,https://en.wikipedia.org/wiki/A-0_System


In [209]:
prog_langs = sc.textFile("/mnt/data/programming-languages.csv").map(lambda x: x.split(","))
prog_langs.take(5)

[['name', 'wikipedia_url'],
 ['A# .NET', 'https://en.wikipedia.org/wiki/A_Sharp_(.NET)'],
 ['A# (Axiom)', 'https://en.wikipedia.org/wiki/A_Sharp_(Axiom)'],
 ['A-0 System', 'https://en.wikipedia.org/wiki/A-0_System'],
 ['A+', 'https://en.wikipedia.org/wiki/A%2B_(programming_language)']]

In [210]:
prog_langs_list = prog_langs.map(lambda x: x[0]).collect()
prog_langs_list[1:5]

['A# .NET', 'A# (Axiom)', 'A-0 System', 'A+']

In [190]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.15.0 pyspark-shell'

posts_sample = spark_session.read.format("xml").options(rowTag="row").load('/mnt/data/posts_sample.xml').rdd
posts_sample.first()

Row(_AcceptedAnswerId=7, _AnswerCount=13, _Body="<p>I want to use a track-bar to change a form's opacity.</p>\n\n<p>This is my code:</p>\n\n<pre><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n\n<p>When I build the application, it gives the following error:</p>\n\n<blockquote>\n  <p>Cannot implicitly convert type <code>'decimal'</code> to <code>'double'</code></p>\n</blockquote>\n\n<p>I tried using <code>trans</code> and <code>double</code> but then the control doesn't work. This code worked fine in a past VB.NET project.</p>\n", _ClosedDate=None, _CommentCount=2, _CommunityOwnedDate=datetime.datetime(2012, 10, 31, 16, 42, 47, 213000), _CreationDate=datetime.datetime(2008, 7, 31, 21, 42, 52, 667000), _FavoriteCount=48, _Id=4, _LastActivityDate=datetime.datetime(2019, 7, 19, 1, 39, 54, 173000), _LastEditDate=datetime.datetime(2019, 7, 19, 1, 39, 54, 173000), _LastEditorDisplayName='Rich B', _LastEditorUserId=3641067, _OwnerDisplayName=None, _OwnerUse

In [191]:
def CheckDateRange(row):
    left_date = datetime(year=2010, month=1, day=1)
    right_date = datetime(year=2019, month=12, day=31)
    return row._CreationDate > left_date and row._CreationDate < right_date

def FindTag(row):
    tag = None
    for lang in prog_langs_list:
        if '<' + lang.lower() + '>' in row._Tags.lower():
            tag = lang
            break
    if tag is None:
        return None
    return (row._Id, row._CreationDate.year, tag)

In [192]:
langs_2010_2020 = posts_sample.filter(lambda row: row._Tags is not None and CheckDateRange(row))\
                        .map(FindTag).filter(lambda row: row is not None)\
                        .keyBy(lambda row: (row[1], row[2]))\
                        .aggregateByKey(0,lambda acc, value: acc + 1,lambda acc1, acc2: acc1 + acc2)\
                        .map(lambda row: (row[0][0], row[0][1], row[1])).toDF(('Year','Language','CountOfPosts'))
langs_2010_2020.show(10,False)

+----+------------+------------+
|Year|Language    |CountOfPosts|
+----+------------+------------+
|2010|Python      |25          |
|2010|JavaScript  |44          |
|2010|R           |3           |
|2011|Objective-C |33          |
|2011|JavaScript  |82          |
|2013|JavaScript  |196         |
|2013|Python      |87          |
|2013|ActionScript|1           |
|2014|JavaScript  |235         |
|2014|Python      |103         |
+----+------------+------------+
only showing top 10 rows



In [193]:
langs_2010_2020.write.mode('overwrite').parquet("parquets/langs_2010_2020")

In [194]:
folder = "parquets/top_langs_2010_2020"
os.makedirs(folder_save_name, exist_ok=True)

parquetFile = spark_session.read.parquet("parquets/langs_2010_2020")
parquetFile.createOrReplaceTempView("parquetFile")
for y in range(2010, 2020):
    print('_____________')
    print('|   ', y, "  |")
    top_10 = spark_session.sql(
        f"select Language from parquetFile where Year = {y} order by CountOfPosts desc limit 10 "
    )
    top_10.write.mode('overwrite').parquet(f"{folder}/top_10_langs_{y}")
    top_10.show()
    

_____________
|    2010   |
+-----------+
|   Language|
+-----------+
|       Java|
| JavaScript|
|        PHP|
|     Python|
|Objective-C|
|          C|
|       Ruby|
|     Delphi|
|       Bash|
|          R|
+-----------+

_____________
|    2011   |
+-----------+
|   Language|
+-----------+
|        PHP|
|       Java|
| JavaScript|
|     Python|
|Objective-C|
|          C|
|       Ruby|
|       Perl|
|     Delphi|
|       Bash|
+-----------+

_____________
|    2012   |
+-----------+
|   Language|
+-----------+
|        PHP|
| JavaScript|
|       Java|
|     Python|
|Objective-C|
|          C|
|       Ruby|
|          R|
|       Bash|
|      Scala|
+-----------+

_____________
|    2013   |
+-----------+
|   Language|
+-----------+
| JavaScript|
|       Java|
|        PHP|
|     Python|
|Objective-C|
|          C|
|       Ruby|
|          R|
|       Bash|
|      Scala|
+-----------+

_____________
|    2014   |
+-----------+
|   Language|
+-----------+
| JavaScript|
|       Java|
| 